# Simple Reinforcement Learning in Tensorflow Part 2: Policy Gradient Method
This tutorial contains a simple example of how to build a policy-gradient based agent that can solve the CartPole problem. For more information, see this [Medium post](https://medium.com/@awjuliani/super-simple-reinforcement-learning-tutorial-part-2-ded33892c724#.mtwpvfi8b).

For more Reinforcement Learning algorithms, including DQN and Model-based learning in Tensorflow, see my Github repo, [DeepRL-Agents](https://github.com/awjuliani/DeepRL-Agents). 

Parts of this tutorial are based on code by [Andrej Karpathy](https://gist.github.com/karpathy/a4166c7fe253700972fcbc77e4ea32c5) and [korymath](https://gym.openai.com/evaluations/eval_a0aVJrGSyW892vBM04HQA).

In [1]:
from __future__ import division

import numpy as np
try:
    import cPickle as pickle
except:
    import pickle
import tensorflow as tf
import tensorflow.contrib.slim as slim
%matplotlib inline
import matplotlib.pyplot as plt
import math

try:
    xrange = xrange
except:
    xrange = range

/home/kimbring2/MineRL/env/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/kimbring2/MineRL/env/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/kimbring2/MineRL/env/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/kimbring2/MineRL/env/lib/python3.6/site-packages/tensorflow/pyth

### Loading the CartPole Environment
If you don't already have the OpenAI gym installed, use  `pip install gym` to grab it.

What happens if we try running the environment with random actions? How well do we do? (Hint: not so well.)

The goal of the task is to achieve a reward of 200 per episode. For every step the agent keeps the pole in the air, the agent recieves a +1 reward. By randomly choosing actions, our reward for each episode is only a couple dozen. Let's make that better with RL!

### Setting up our Neural Network agent
This time we will be using a Policy neural network that takes observations, passes them through a single hidden layer, and then produces a probability of choosing a left/right movement. To learn more about this network, see [Andrej Karpathy's blog on Policy Gradient networks](http://karpathy.github.io/2016/05/31/rl/).

In [2]:
# hyperparameters
H = 57600 # number of hidden layer neurons
batch_size = 5 # every how many episodes to do a param update?
learning_rate = 1e-2 # feel free to play with this to train faster or more stably.
gamma = 0.99 # discount factor for reward

D = 4 # input dimensionality

In [3]:
tf.reset_default_graph()

#This defines the network as it goes from taking an observation of the environment to 
#giving a probability of chosing to the action of moving left or right.
state = tf.placeholder(shape=[None,64,64,4], dtype=tf.float32)
conv1 = slim.conv2d( \
            inputs=state,num_outputs=16,kernel_size=[2,2],stride=[1,1],padding='VALID', 
                    biases_initializer=None,activation_fn=tf.nn.relu)
conv2 = slim.conv2d( \
            inputs=conv1,num_outputs=16,kernel_size=[2,2],stride=[1,1],padding='VALID', 
                    biases_initializer=None,activation_fn=tf.nn.relu)
conv3 = slim.conv2d( \
            inputs=conv2,num_outputs=16,kernel_size=[2,2],stride=[1,1],padding='VALID', 
                    biases_initializer=None,activation_fn=tf.nn.relu)
conv4 = slim.conv2d( \
            inputs=conv3,num_outputs=16,kernel_size=[2,2],stride=[1,1],padding='VALID', 
                    biases_initializer=None,activation_fn=tf.nn.relu)

convFlat = slim.flatten(conv4)
print("convFlat: " + str(convFlat))

#observations = tf.placeholder(tf.float32, [None,D] , name="input_x")
W = tf.get_variable("W", shape=[H, 3],
           initializer=tf.contrib.layers.xavier_initializer())
score = tf.matmul(convFlat, W)
probability = tf.nn.softmax(score)

#W_Left = tf.get_variable("W_Left", shape=[H, 2],
#           initializer=tf.contrib.layers.xavier_initializer())
#score_Left = tf.matmul(convFlat, W_Left)
#probability_Left = tf.nn.softmax(score_Left)

#W_Jump = tf.get_variable("W_Jump", shape=[H, 2],
#           initializer=tf.contrib.layers.xavier_initializer())
#score_Jump = tf.matmul(convFlat, W_Jump)
#probability_Jump = tf.nn.softmax(score_Jump)

#W_Right = tf.get_variable("W_Right", shape=[H, 2],
#           initializer=tf.contrib.layers.xavier_initializer())
#score_Right = tf.matmul(convFlat, W_Right)
#probability_Right = tf.nn.softmax(W_Right)

#From here we define the parts of the network needed for learning a good policy.
#tvars = tf.trainable_variables()
#input_y = tf.placeholder(tf.float32, [None,1], name="input_y")
#advantages = tf.placeholder(tf.float32, name="reward_signal")

real_action = tf.placeholder(shape=[None, 3], dtype=tf.int32)
#left_real_action = tf.placeholder(shape=[None, 2], dtype=tf.int32)
#jump_real_action = tf.placeholder(shape=[None, 2], dtype=tf.int32)
#right_real_action = tf.placeholder(shape=[None, 2], dtype=tf.int32)

# The loss function. This sends the weights in the direction of making actions 
# that gave good advantage (reward over time) more likely, and actions that didn't less likely.
#loglik = tf.log(input_y*(input_y - probability) + (1 - input_y)*(input_y + probability))
#loss = -tf.reduce_mean(loglik * advantages) 
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=real_action, 
                                                                     logits=score))
#loss_left = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=left_real_action, 
#                                                                           logits=score_Left))
#loss_jump = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=jump_real_action, 
#                                                                        logits=score_Jump))
#loss_right = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=right_real_action, 
#                                                                          logits=score_Right))
#loss = loss_back + loss_forward + loss_jump + loss_attack
tf.summary.scalar('loss', loss)
train_step = tf.train.RMSPropOptimizer(0.001).minimize(loss)

# Merge all the summaries and write them out to /tmp/mnist_logs (by default)
merged = tf.summary.merge_all()

'''
newGrads = tf.gradients(loss, tvars)

# Once we have collected a series of gradients from multiple episodes, we apply them.
# We don't just apply gradeients after every episode in order to account for noise in the reward signal.
adam = tf.train.AdamOptimizer(learning_rate=learning_rate) # Our optimizer
W1Grad = tf.placeholder(tf.float32,name="batch_grad1") # Placeholders to send the final gradients through when we update.
W2Grad = tf.placeholder(tf.float32,name="batch_grad2")
batchGrad = [W1Grad,W2Grad]
updateGrads = adam.apply_gradients(zip(batchGrad,tvars))
'''

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.flatten instead.
convFlat: Tensor("Flatten/flatten/Reshape:0", shape=(?, 57600), dtype=float32)
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



'\nnewGrads = tf.gradients(loss, tvars)\n\n# Once we have collected a series of gradients from multiple episodes, we apply them.\n# We don\'t just apply gradeients after every episode in order to account for noise in the reward signal.\nadam = tf.train.AdamOptimizer(learning_rate=learning_rate) # Our optimizer\nW1Grad = tf.placeholder(tf.float32,name="batch_grad1") # Placeholders to send the final gradients through when we update.\nW2Grad = tf.placeholder(tf.float32,name="batch_grad2")\nbatchGrad = [W1Grad,W2Grad]\nupdateGrads = adam.apply_gradients(zip(batchGrad,tvars))\n'

### Advantage function
This function allows us to weigh the rewards our agent recieves. In the context of the Cart-Pole task, we want actions that kept the pole in the air a long time to have a large reward, and actions that contributed to the pole falling to have a decreased or negative reward. We do this by weighing the rewards from the end of the episode, with actions at the end being seen as negative, since they likely contributed to the pole falling, and the episode ending. Likewise, early actions are seen as more positive, since they weren't responsible for the pole falling.

In [4]:
#!pip install 'tensorboard==1.0.0a6'

In [5]:
def discount_rewards(r):
    """ take 1D float array of rewards and compute discounted reward """
    discounted_r = np.zeros_like(r)
    running_add = 0
    for t in reversed(xrange(0, r.size)):
        running_add = running_add * gamma + r[t]
        discounted_r[t] = running_add
    return discounted_r

### Running the Agent and Environment

Here we run the neural network agent, and have it act in the CartPole environment.

In [6]:
'''
import minerl
import gym
env = gym.make('MineRLNavigateDense-v0')

obs  = env.reset()
done = False
net_reward = 0

while not done:
    action = env.action_space.noop()

    action['camera'] = [0, -10]
    action['back'] = 0
    action['forward'] = 1
    action['jump'] = 1
    action['attack'] = 1

    obs, reward, done, info = env.step(
        action)

    net_reward += reward
    print("Total reward: ", net_reward)
'''

'\nimport minerl\nimport gym\nenv = gym.make(\'MineRLNavigateDense-v0\')\n\nobs  = env.reset()\ndone = False\nnet_reward = 0\n\nwhile not done:\n    action = env.action_space.noop()\n\n    action[\'camera\'] = [0, -10]\n    action[\'back\'] = 0\n    action[\'forward\'] = 1\n    action[\'jump\'] = 1\n    action[\'attack\'] = 1\n\n    obs, reward, done, info = env.step(\n        action)\n\n    net_reward += reward\n    print("Total reward: ", net_reward)\n'

In [7]:
#import minerl
#data = minerl.data.make('MineRLNavigateDense-v0', '/home/kimbring2/MineRL/data/')

In [8]:

import minerl
import gym

env = gym.make('MineRLNavigateDense-v0')
obs = env.reset()


In [9]:
xs,hs,dlogps,drs,ys,tfps = [],[],[],[],[],[]
running_reward = None
reward_sum = 0
episode_number = 1
total_episodes = 10000
init = tf.global_variables_initializer()
with tf.Session() as sess:

# Launch the graph
    rendering = False
    sess.run(init)
    saver = tf.train.Saver(max_to_keep=5)
    train_writer = tf.summary.FileWriter('/home/kimbring2/MineRL/train_summary', sess.graph)
    
    print('Loading Model...')
    path = '/home/kimbring2/MineRL/model'
    ckpt = tf.train.get_checkpoint_state(path)
    saver.restore(sess, ckpt.model_checkpoint_path)
    
    env.init()
    obs = env.reset()
    net_reward = 0
    for i in range(0, 500000):
        pov = obs['pov'].astype(np.float32) / 255.0 - 0.5
        compass = obs['compassAngle']

        compass_channel = np.ones(shape=list(pov.shape[:-1]) + [1], dtype=np.float32) * compass
        compass_channel /= 180.0
        
        state_concat = np.concatenate([pov, compass_channel], axis=-1)
        #print("state_concat.shape: " + str(state_concat.shape))
        
        #action = env.action_space.noop()
        #observations = tf.placeholder(tf.float32, [None,D] , name="input_x")
        action_probability = sess.run(probability, feed_dict={state:[state_concat]})
        #l = sess.run(probability_Left, feed_dict={state:[obs['pov']]})
        #j = sess.run(probability_Jump, feed_dict={state:[obs['pov']]})
        #r =  sess.run(probability_Right, feed_dict={state:[obs['pov']]})
        
        #print("c: " + str(c))
        #print("np.argmax(b): " + str(np.argmax(b)))
        #print("np.argmax(f): " + str(np.argmax(f)))
        #print("np.argmax(j): " + str(np.argmax(j)))
        #print("np.argmax(a): " + str(np.argmax(a)))
        
        action = env.action_space.noop()
        if (np.argmax(action_probability) == 0):
            action['camera'] = [0, -10]
            action['jump'] = 0
        elif (np.argmax(action_probability) == 1):
            action['camera'] = [0, 10]
            action['jump'] = 0
        else:
            action['camera'] = [0, 0]
            action['jump'] = 1
        
        action['forward'] = 1
        action['back'] = 0
        action['left'] = 0
        #action['jump'] = np.argmax(j)
        action['right'] = 0
        action['sprint'] = 1

        obs, reward, done, info = env.step(action)
        
        if done == True:
            break

        net_reward += reward
        print("Total reward: ", net_reward)
    '''
    
    episode_count = 0
    for current_state, action, reward, next_state, done in data.sarsd_iter(num_epochs=500, max_sequence_len=200):
        #print("current_state['compassAngle']: " + str(current_state['compassAngle']))
        #print("action: " + str(action))
        #print("reward: " + str(reward))
        length = (current_state['pov'].shape)[0]

        #print("state_concat.shape: " + str(state_concat.shape))

        action_list = []
        states_list = []
        #camera_action_list = []
        #left_action_list = []
        #jump_action_list = []
        #right_action_list = []
        for i in range(0, length):
            #states = current_state['pov'][i]
            pov = current_state['pov'][i].astype(np.float32) / 255.0 - 0.5
            compass = current_state['compassAngle'][i]

            compass_channel = np.ones(shape=list(pov.shape[:-1]) + [1], dtype=np.float32) * compass
            compass_channel /= 180.0
        
            state_concat = np.concatenate([pov, compass_channel], axis=-1)
            #print("state_concat.shape: " + str(state_concat.shape))
            
            #print("action['camera'][i]: " + str(action['camera'][i]))
            #print("action['camera'][i][1]: " + str(action['camera'][i][1]))
            #print("")
            
            if (action['camera'][i][1] < 0):
                action_ = [1, 0, 0]
            elif (action['camera'][i][1] > 0):
                action_ = [0, 1, 0]
            else:
                action_ = [0, 0, 1]
                
            #camera_action = np.eye(2)[action['camera'][i]]
            #left_action = np.eye(2)[action['left'][i]]
            #jump_action = np.eye(2)[action['jump'][i]]
            #right_action = np.eye(2)[action['right'][i]]
            
            states_list.append(state_concat)
            action_list.append(action_)
            #left_action_list.append(left_action)
            #jump_action_list.append(jump_action)
            #right_action_list.append(right_action)
        
        episode_count = episode_count + 1
        
        #while True:
        feed_dict = {state:np.stack(states_list, 0),
                     real_action:np.stack(action_list, 0)
                    }
    
        if episode_count % 100 == 0:
            #print("loss_print: " + str(loss_print))
            #tf.summary.scalar('loss', loss)
            summary, _ = sess.run([merged, train_step], feed_dict=feed_dict)
            train_writer.add_summary(summary, episode_count)

        sess.run(train_step, feed_dict=feed_dict)
        
        if episode_count % 100 == 0:
            model_path = '/home/kimbring2/MineRL/model'
            saver.save(sess, model_path + '/model-' + str(episode_count) + '.cptk')
            print("Saved Model")
    '''
print(episode_number, 'Episodes completed.')

Loading Model...
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from /home/kimbring2/MineRL/model/model-60600.cptk
Total reward:  0.13396453857421875
Total reward:  0.20639801025390625
Total reward:  0.295379638671875
Total reward:  0.40094757080078125
Total reward:  0.5258026123046875
Total reward:  0.671539306640625
Total reward:  0.8387222290039062
Total reward:  1.0260086059570312
Total reward:  1.2327880859375
Total reward:  1.4574813842773438
Total reward:  1.6982421875
Total reward:  1.9146957397460938
Total reward:  2.3700637817382812
Total reward:  2.6284027099609375
Total reward:  2.8933868408203125
Total reward:  3.1661453247070312
Total reward:  3.44744873046875
Total reward:  3.7377548217773438
Total reward:  4.0355224609375
Total reward:  4.340732574462891
Total reward:  4.624519348144531
Total reward:  5.105266571044922
Total reward:  5.3719635009765625
Total reward:  5.64349365234375
Total rewa

Total reward:  60.43043231964111
Total reward:  60.52682113647461
Total reward:  60.62146520614624
Total reward:  60.714346408843994
Total reward:  60.80543088912964
Total reward:  60.90446376800537
Total reward:  61.0266695022583
Total reward:  61.169591426849365
Total reward:  61.32028675079346
Total reward:  61.45692300796509
Total reward:  61.5836763381958
Total reward:  61.703227519989014
Total reward:  61.81728649139404
Total reward:  61.92693614959717
Total reward:  62.03285789489746
Total reward:  62.135475158691406
Total reward:  62.23504400253296
Total reward:  62.331714153289795
Total reward:  62.425562381744385
Total reward:  62.51662063598633
Total reward:  62.60488557815552
Total reward:  62.70141935348511
Total reward:  62.82324266433716
Total reward:  62.9679160118103
Total reward:  63.12072563171387
Total reward:  63.25766372680664
Total reward:  63.383572578430176
Total reward:  63.50149869918823
Total reward:  63.613319396972656
Total reward:  63.7201566696167
Total 

Total reward:  47.28474235534668
Total reward:  47.1906795501709
Total reward:  47.0982608795166
Total reward:  47.00704383850098
Total reward:  46.91667175292969
Total reward:  46.82685470581055
Total reward:  46.73736381530762
Total reward:  46.64425468444824
Total reward:  46.539791107177734
Total reward:  46.42466163635254
Total reward:  46.30306625366211
Total reward:  46.187660217285156
Total reward:  46.07768249511719
Total reward:  45.9722843170166
Total reward:  45.87063407897949
Total reward:  45.77198028564453
Total reward:  45.6756706237793
Total reward:  45.58115196228027
Total reward:  45.487966537475586
Total reward:  45.3957405090332
Total reward:  45.30417060852051
Total reward:  45.213016510009766
Total reward:  45.12208366394043
Total reward:  45.02727699279785
Total reward:  44.920658111572266
Total reward:  44.80302429199219
Total reward:  44.67885780334473
Total reward:  44.56112480163574
Total reward:  44.448970794677734
Total reward:  44.34148979187012
Total rew

Total reward:  34.53849792480469
Total reward:  34.623512268066406
Total reward:  34.70937728881836
Total reward:  34.79584503173828
Total reward:  34.88276672363281
Total reward:  34.97002029418945
Total reward:  35.057464599609375
Total reward:  35.14499282836914
Total reward:  35.23250961303711
Total reward:  35.319942474365234
Total reward:  35.4072265625
Total reward:  35.48896408081055
Total reward:  35.56720733642578
Total reward:  35.65470886230469
Total reward:  35.75718688964844
Total reward:  35.87441635131836
Total reward:  36.005653381347656
Total reward:  36.14967346191406
Total reward:  36.30477714538574
Total reward:  36.466867446899414
Total reward:  36.86198806762695
Total reward:  37.08930778503418
Total reward:  37.316471099853516
Total reward:  37.54555702209473
Total reward:  37.77796173095703
Total reward:  38.01442337036133
Total reward:  38.25503730773926
Total reward:  38.499284744262695
Total reward:  38.74603843688965
Total reward:  38.99358367919922
Total r

Total reward:  59.12689018249512
Total reward:  59.22970008850098
Total reward:  59.327423095703125
Total reward:  59.42101001739502
Total reward:  59.51109027862549
Total reward:  59.5980863571167
Total reward:  59.682281494140625
Total reward:  59.76386833190918
Total reward:  59.84298133850098
Total reward:  59.91970920562744
Total reward:  59.99411201477051
Total reward:  60.07463073730469
Total reward:  60.17414665222168
Total reward:  60.29038858413696
Total reward:  60.41213035583496
Total reward:  60.521077156066895
Total reward:  60.62089776992798
Total reward:  60.71393156051636
Total reward:  60.801663398742676
Total reward:  60.885032176971436
Total reward:  60.96462869644165
Total reward:  61.0408239364624
Total reward:  61.11384868621826
Total reward:  61.1838436126709
Total reward:  61.250892639160156
Total reward:  61.3150429725647
Total reward:  61.38451814651489
Total reward:  61.47157287597656
Total reward:  61.57360124588013
Total reward:  61.67920637130737
Total re

...

Total reward:  7.578517913818359
Total reward:  7.544658660888672
Total reward:  7.511135101318359
Total reward:  7.477813720703125
Total reward:  7.444599151611328
Total reward:  7.411403656005859
Total reward:  7.378154754638672
Total reward:  7.343402862548828
Total reward:  7.3042449951171875
Total reward:  7.26092529296875
Total reward:  7.214996337890625
Total reward:  7.17120361328125
Total reward:  7.129268646240234
Total reward:  7.088890075683594
Total reward:  7.049770355224609
Total reward:  7.011634826660156
Total reward:  6.9742584228515625
Total reward:  6.937431335449219
Total reward:  6.900993347167969
Total reward:  6.864803314208984
Total reward:  6.8287506103515625
Total reward:  6.792747497558594
Total reward:  6.755130767822266
Total reward:  6.712650299072266
Total reward:  6.665576934814453
Total reward:  6.615684509277344
Total reward:  6.568241119384766
Total reward:  6.522911071777344
Total reward:  6.479343414306641
Total reward:  6.437202453613281
Total rew

Total reward:  -6.4927215576171875
Total reward:  -6.573768615722656
Total reward:  -6.6513214111328125
Total reward:  -6.7260589599609375
Total reward:  -6.798561096191406
Total reward:  -6.869293212890625
Total reward:  -6.938636779785156
Total reward:  -7.006927490234375
Total reward:  -7.074394226074219
Total reward:  -7.141265869140625
Total reward:  -7.2076873779296875
Total reward:  -7.2772369384765625
Total reward:  -7.3564453125
Total reward:  -7.4445648193359375
Total reward:  -7.537513732910156
Total reward:  -7.625030517578125
Total reward:  -7.70806884765625
Total reward:  -7.7874603271484375
Total reward:  -7.863914489746094
Total reward:  -7.9380340576171875
Total reward:  -8.010299682617188
Total reward:  -8.08111572265625
Total reward:  -8.15081787109375
Total reward:  -8.219657897949219
Total reward:  -8.287864685058594
Total reward:  -8.3555908203125
Total reward:  -8.422988891601562
Total reward:  -8.493667602539062
Total reward:  -8.574256896972656
Total reward:  -

Total reward:  11.714614868164062
Total reward:  11.81436538696289
Total reward:  11.913902282714844
Total reward:  12.013233184814453
Total reward:  12.11236572265625
Total reward:  12.2113037109375
Total reward:  12.310047149658203
Total reward:  12.408599853515625
Total reward:  12.506961822509766
Total reward:  12.60513687133789
Total reward:  12.703125
Total reward:  12.800926208496094
Total reward:  12.898536682128906
Total reward:  12.995964050292969
Total reward:  13.093204498291016
Total reward:  13.190258026123047
Total reward:  13.287120819091797
Total reward:  13.383796691894531
Total reward:  13.486675262451172
Total reward:  13.606624603271484
Total reward:  13.742095947265625
Total reward:  13.885169982910156
Total reward:  14.018814086914062
Total reward:  14.145061492919922
Total reward:  14.265472412109375
Total reward:  14.381237030029297
Total reward:  14.493278503417969
Total reward:  14.602313995361328
Total reward:  14.708915710449219
Total reward:  14.8135299682

Total reward:  40.50237846374512
Total reward:  40.606590270996094
Total reward:  40.72846603393555
Total reward:  40.866294860839844
Total reward:  41.011451721191406
Total reward:  41.14618110656738
Total reward:  41.27292442321777
Total reward:  41.39345359802246
Total reward:  41.509084701538086
Total reward:  41.62079429626465
Total reward:  41.72933006286621
Total reward:  41.83525848388672
Total reward:  41.93902397155762
Total reward:  42.04097366333008
Total reward:  42.14137649536133
Total reward:  42.240447998046875
Total reward:  42.338361740112305
Total reward:  42.442230224609375
Total reward:  42.56398963928223
Total reward:  42.70190620422363
Total reward:  42.84720230102539
Total reward:  42.9819278717041
Total reward:  43.10857582092285
Total reward:  43.22894859313965
Total reward:  43.34437561035156
Total reward:  43.45584487915039
Total reward:  43.56410598754883
Total reward:  43.669729232788086
Total reward:  43.773155212402344
Total reward:  43.87472915649414
To

Total reward:  63.022048473358154
Total reward:  62.95161056518555
Total reward:  62.880088806152344
Total reward:  62.80718994140625
Total reward:  62.73267889022827
Total reward:  62.65638256072998
Total reward:  62.578182220458984
Total reward:  62.49879789352417
Total reward:  62.41623544692993
Total reward:  62.329118728637695
Total reward:  62.234981060028076
Total reward:  62.14000654220581
Total reward:  62.04623889923096
Total reward:  61.954336643218994
Total reward:  61.8676872253418
Total reward:  61.7841010093689
Total reward:  61.70299577713013
Total reward:  61.62367582321167
Total reward:  61.54553031921387
Total reward:  61.47200918197632
Total reward:  61.40506076812744
Total reward:  61.34622812271118
Total reward:  61.29674959182739
Total reward:  61.25357532501221
Total reward:  61.215312480926514
Total reward:  61.18063306808472
Total reward:  61.14840126037598
Total reward:  61.1175742149353
Total reward:  61.08731555938721
Total reward:  61.05694341659546
Total 

As you can see, the network not only does much better than random actions, but achieves the goal of 200 points per episode, thus solving the task!